In [109]:
import pandas as pd
import numpy as np
from collections import Counter

In [110]:
file_path = 'Bank.xlsx'
df = pd.read_excel(file_path)

df.columns = df.columns.str.strip()

df = df.dropna(subset=['Sense of the word "Bank"'])
df = df[df['Sense of the word "Bank"'] != '?']

In [111]:
training_data=list(zip(df['Sentences for Training'],df['Sense of the word "Bank"']))

In [112]:
word_counts={
    "Financial Institution":defaultdict(int),
    "River Border":defaultdict(int)
}
sense_counts=defaultdict(int)
total_words=0

In [113]:
for sentence, sense in training_data:
    words = sentence.lower().split()
    sense_counts[sense] += 1
    for word in words:
        word_counts[sense][word] += 1
        total_words += 1

In [114]:
total_sentences=len(training_data)
total_sentences

93

In [115]:
total_words

972

In [116]:
prior_prob={
    sense:count/total_sentences for sense,count in sense_counts.items()
}
prior_prob

{'Financial Institution': 0.4838709677419355,
 'River Border': 0.5161290322580645}

In [117]:
vocabulary=set(word for sense in word_counts for word in word_counts[sense])
v_size=len(vocabulary)

In [118]:
conditional_prob={
    sense:{
        word:(word_counts[sense][word]+1)/(sum(word_counts[sense].values())+v_size)
        for word in vocabulary
    }
    for sense in word_counts
}

In [119]:
import math
def classify(sentence):
    words = sentence.lower().split()
    scores = {}
    for sense in prior_prob:
        score = math.log(prior_prob[sense])
        for word in words:
            if sense in ["Financial Institution","River Border"]:
                if word in conditional_prob[sense]:
                    score += math.log(conditional_prob[sense][word])
                else:
                    score += math.log(1 / (sum(word_counts[sense].values()) + v_size))
        scores[sense] = score
    return max(scores, key=scores.get)

In [120]:
test

,Unnamed: 0,Sentences for Training,"Sense of the word ""Bank"""
94,NaN,The children built a dam on the bank of the ri...,River Border
95,NaN,We need to withdraw some cash from the bank fo...,Financial Institution
96,NaN,I need to update my contact information with t...,Financial Institution
97,NaN,The bank provides online banking services for ...,Financial Institution
98,NaN,The beavers constructed a dam along the bank o...,River Border
99,NaN,I need to check my transaction history at the ...,Financial Institution
100,NaN,She works as a financial consultant at the bank.,Financial Institution


In [107]:
for sentence in test['Sentences for Training']:
    test['Sense of the word "Bank"']=test['Sentences for Training'].apply(classify)

test

,Unnamed: 0,Sentences for Training,"Sense of the word ""Bank"""
94,NaN,The children built a dam on the bank of the ri...,River Border
95,NaN,We need to withdraw some cash from the bank fo...,Financial Institution
96,NaN,I need to update my contact information with t...,Financial Institution
97,NaN,The bank provides online banking services for ...,Financial Institution
98,NaN,The beavers constructed a dam along the bank o...,River Border
99,NaN,I need to check my transaction history at the ...,Financial Institution
100,NaN,She works as a financial consultant at the bank.,Financial Institution
